In [16]:
import os
import shutil
import json
import numpy as np
from PIL import Image

In [2]:
# WAD database
DATABASE_FOLDER = '..\\data'
ANNOTATION_FILE = os.path.join(DATABASE_FOLDER, 'wider_attribute_annotation\\wider_attribute_trainval.json')
IMAGES_FOLDER = os.path.join(DATABASE_FOLDER, 'wider_attribute_image\\Image')

In [3]:
SSD_DATA = '..\\data\\ssd_data'
TRAIN_IMAGES_FOLDER = os.path.join(SSD_DATA, 'images')
TRAIN_XML_FOLDER = os.path.join(SSD_DATA, 'xml')
TEST_IMAGES_FOLDER = os.path.join(SSD_DATA, 'test_images')
TEST_XML_FOLDER = os.path.join(SSD_DATA, 'test_xml')
TRAIN_TXT = os.path.join(SSD_DATA, 'train.txt')
TEST_TXT = os.path.join(SSD_DATA, 'test.txt')

In [4]:
# Creating selection folders with data and its marking
if not os.path.exists(SSD_DATA):
    os.makedirs(SSD_DATA)
if not os.path.exists(TRAIN_IMAGES_FOLDER):
    os.makedirs(TRAIN_IMAGES_FOLDER)
if not os.path.exists(TRAIN_XML_FOLDER):
    os.makedirs(TRAIN_XML_FOLDER)
if not os.path.exists(TEST_IMAGES_FOLDER):
    os.makedirs(TEST_IMAGES_FOLDER)
if not os.path.exists(TEST_XML_FOLDER):
    os.makedirs(TEST_XML_FOLDER)

In [5]:
with open(ANNOTATION_FILE) as f:
    annotation_file = json.load(f)

In [6]:
def get_gender(label):
    if (label == -1):
        return 'f'
    if (label == 1):
        return 'm'
    if (label == 0):
        return 'u'
    return 'u'

In [7]:
# dict of images, bboxes and labels
ssd_dict = dict()

In [8]:
# fill ssd dict
for anno in annotation_file['images']: # each element - dict
    pict = anno['file_name']
    for targ in anno['targets']: # each element - dict
        gend = get_gender(targ['attribute'][0]) # label
        bbox = list(map(int, targ['bbox'])) 
        frame = [bbox[0]+1, bbox[1]+1, bbox[0]+bbox[2]-1, bbox[1]+bbox[3]-1]
        if gend != 'u':
            ssd_content = [frame, gend]
            if not (pict in ssd_dict.keys()):
                ssd_dict[pict] = [ssd_content]
            else:
                ssd_dict[pict].append(ssd_content)

In [9]:
# some important sections for xml marking
DB_NAME = 'UAD'
IMG_TYPE = 'gender'
CHANNELS = 3

In [10]:
# create the conthent of xml elements with bboxes and classes
def xml_objects(objs):
    res = ""
    for el in objs:
        res += '  <object>\n' + \
        '    <name>{}</name>\n'.format(el[1]) + \
        '    <bndbox>\n' + \
        '      <xmin>{}</xmin>\n'.format(el[0][0]) + \
        '      <ymin>{}</ymin>\n'.format(el[0][1]) + \
        '      <xmax>{}</xmax>\n'.format(el[0][2]) + \
        '      <ymax>{}</ymax>\n'.format(el[0][3]) + \
        '    </bndbox>\n' + \
        '  </object>\n'
    return res

In [11]:
# create the xml content for image
def xml_content(folder_name, file_name, db_name, selection_type, img_type, width, height, channels, voc_el):
    xml_objs = xml_objects(voc_el)
    if (xml_objs != ''):
        return  '<annotation>\n' + \
                '  <folder>{}</folder>\n'.format(folder_name) + \
                '  <filename>{}</filename>\n'.format(file_name) + \
                '  <source>\n' + \
                '    <database>{}</database>\n'.format(db_name) + \
                '    <annotation>{}</annotation>\n'.format(selection_type) + \
                '    <image>{}</image>\n'.format(img_type) + \
                '  </source>\n' + \
                '  <size>\n' + \
                '    <width>{}</width>\n'.format(width) + \
                '    <height>{}</height>\n'.format(height) + \
                '    <depth>{}</depth>\n'.format(channels) + \
                '  </size>\n' + xml_objs + '</annotation>'

In [12]:
def make_xml(xml_folder_name, xml_file, img_folder_name, img_file, db_name, annotation_type, img_type, width, height, channels, voc_el):
    with open(os.path.join(xml_folder_name, xml_file), 'w') as output_file:
        str_data = xml_content(img_folder_name, img_file, db_name, annotation_type, img_type, width, height, channels, voc_el)
        output_file.write(str_data)

In [13]:
ssd_dict

{'train/0--Parade/0_Parade_marchingband_1_100.jpg': [[[712, 209, 1018, 655],
   'm'],
  [[746, 181, 844, 396], 'f'],
  [[394, 135, 488, 578], 'm'],
  [[482, 127, 568, 593], 'm'],
  [[255, 116, 409, 653], 'm'],
  [[181, 83, 296, 619], 'm']],
 'train/0--Parade/0_Parade_marchingband_1_1015.jpg': [[[4, 27, 1017, 656],
   'm']],
 'train/0--Parade/0_Parade_marchingband_1_1018.jpg': [[[351, 241, 537, 673],
   'm']],
 'train/0--Parade/0_Parade_marchingband_1_1022.jpg': [[[539, 259, 728, 663],
   'm'],
  [[674, 158, 963, 661], 'm'],
  [[452, 316, 584, 660], 'm'],
  [[395, 351, 479, 621], 'f']],
 'train/0--Parade/0_Parade_marchingband_1_1030.jpg': [[[900, 95, 984, 304],
   'f'],
  [[820, 87, 883, 306], 'm'],
  [[682, 86, 769, 339], 'f'],
  [[343, 62, 406, 263], 'm'],
  [[113, 49, 170, 251], 'f'],
  [[28, 65, 90, 227], 'f']],
 'train/0--Parade/0_Parade_marchingband_1_1046.jpg': [[[712, 394, 878, 952],
   'f'],
  [[181, 436, 326, 930], 'f']],
 'train/0--Parade/0_Parade_marchingband_1_1047.jpg': [[

In [17]:
# shuffle dict of ssd data
def shuffle_dict(ssd_data):
    res = dict()
    keys_list = list(ssd_data.keys())
    np.random.seed(42)
    np.random.shuffle(keys_list)
    for el in keys_list:
        res[el] = ssd_data[el]
    return res

In [18]:
ssd_dict = shuffle_dict(ssd_dict)

In [19]:
ssd_dict

{'train/58--Hockey/58_Hockey_icehockey_puck_58_64.jpg': [[[279, 221, 354, 513],
   'm'],
  [[535, 383, 687, 679], 'm']],
 'train/0--Parade/0_Parade_marchingband_1_665.jpg': [[[167, 81, 355, 527],
   'm'],
  [[402, 106, 566, 531], 'm'],
  [[618, 150, 772, 549], 'm'],
  [[840, 131, 1005, 557], 'm']],
 'val/50--Celebration_Or_Party/50_Celebration_Or_Party_houseparty_50_595.jpg': [[[5,
    96,
    516,
    758],
   'm'],
  [[509, 116, 1018, 760], 'm']],
 'train/4--Dancing/4_Dancing_Dancing_4_835.jpg': [[[503, 13, 1015, 660], 'm'],
  [[164, 39, 705, 659], 'f']],
 'train/36--Football/36_Football_americanfootball_ball_36_890.jpg': [[[455,
    39,
    997,
    558],
   'm']],
 'train/25--Soldier_Patrol/25_Soldier_Patrol_Soldier_Patrol_25_643.jpg': [[[5,
    133,
    393,
    564],
   'm']],
 'train/11--Meeting/11_Meeting_Meeting_11_Meeting_Meeting_11_184.jpg': [[[464,
    73,
    1014,
    666],
   'm'],
  [[66, 81, 483, 528], 'f']],
 'train/25--Soldier_Patrol/25_Soldier_Patrol_Soldier_Patrol_

In [20]:
# ratio for extracting train and test data for ssd dict
ratio = 0.75

In [21]:
def train_test_split(ssd_data):
    train_data = dict(list(ssd_data.items())[:int(ratio*len(ssd_data))])
    test_data = dict(list(ssd_data.items())[int(ratio*len(ssd_data)):])
    return train_data, test_data

In [22]:
train_data, test_data = train_test_split(ssd_dict)

In [23]:
# create selection
# param 'selection_type' must be 'train' or 'test'
def make_selection(data, selection_type):
    selection_dict = dict() # data for txt,  which will be used by Caffe for creating lmdb
    # some variables for data, depending on type of selection 
    if (selection_type == 'test'):
        images_folder_name = TEST_IMAGES_FOLDER.split('\\')[-1]
        xml_folder_name = TEST_XML_FOLDER.split('\\')[-1]
        dest_folders = [TEST_XML_FOLDER, TEST_IMAGES_FOLDER]
        txt_file = TEST_TXT
    else:
        images_folder_name = TRAIN_IMAGES_FOLDER.split('\\')[-1]
        xml_folder_name = TRAIN_XML_FOLDER.split('\\')[-1]
        dest_folders = [TRAIN_XML_FOLDER, TRAIN_IMAGES_FOLDER]
        txt_file = TRAIN_TXT
    for img_file in data:
        pict_file = img_file.split('/')[-1]
        voc_el = data[img_file]
        img = Image.open(os.path.join(IMAGES_FOLDER, img_file))
        width, height = img.size
        # xml marking
        xml_file = pict_file.split('.')[0] + '.xml'
        make_xml(dest_folders[0], xml_file, images_folder_name, pict_file, DB_NAME, 
                 selection_type, IMG_TYPE, width, height, CHANNELS, voc_el)
        # image
        shutil.copyfile(os.path.join(IMAGES_FOLDER, img_file), os.path.join(dest_folders[1], pict_file))
        selection_dict[images_folder_name + '/' + pict_file] = xml_folder_name + '/' + xml_file
    # txt for lmdb
    with open(txt_file, 'w+') as output_file:
        for k in selection_dict:
            output_file.write(k + ' ' + selection_dict[k] + '\n')

In [24]:
make_selection(train_data, 'train')

In [25]:
make_selection(test_data, 'test')